In [379]:
import numpy as np
import matplotlib.pyplot as plt

In [520]:
# n = 3
np.random.seed(seed = 1)

length, n = 500, 3
A, P, Sigma = None, None, None 

if n == 2:
    A = np.array([[0.4, 0.9], [0.1, 0.4]])
    P = np.array([[1.0, 0.0], [0.0, 1.0]])
    Sigma = np.identity(n)
elif n == 3:
    A = np.array([[0.9, 0.0, 0.0], [0.9, 0.0, 0.0], [0.0, 0.9, 0.0]])
    Sigma = np.array([[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0]])
    P = np.array([[0.0, 1.0, 0.0], [0.0, 0.0, 1.0], [1.0, 0.0, 0.0]])
elif n == 4:
    A = np.array([[0.9, 0.0, 0.0, 0.0], [0.9, 0.0, 0.0, 0.0], [0.0, 0.9, 0.0, 0.0], [0.0, 0.0, 0.9, 0.0]])
    Sigma = np.identity(n)
    P = np.array([[0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 1.0], [1.0, 0.0, 0.0, 0.0]])
else:
    A = np.zeros((n, n))
    A[0][0] = 0.9
    for i in range(1, n):
        A[i][i - 1] = 0.9
    P = np.identity(n)
    Sigma = np.identity(n)
    
As, Ps = A.copy(), P.copy()

a_parms = int(n * (n + 1) / 2)
p_parms = np.math.factorial(n)

print(A, P)

[[0.9 0.  0. ]
 [0.9 0.  0. ]
 [0.  0.9 0. ]] [[0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]]


In [522]:
def expected_cost(A, P, As = As, Ps = Ps):
    # base on the distribution of X, no actual data needed.
    # we need the covariance of X_t - X_{t-1}.
    # Then, the expected cost is the trace of this covariance
    P_inv = np.linalg.inv(P)
    Ps_inv = np.linalg.inv(Ps)
    
    B = np.matmul(P_inv, np.matmul(A, P))
    Bs = np.matmul(Ps_inv, np.matmul(As, Ps))
    
    covariance_X = np.matmul(np.linalg.inv(np.identity(n ** 2) - np.kron(Bs, Bs)), Sigma.reshape(n ** 2)).reshape((n, n))
    
    covariance_matrix = Sigma + np.matmul((Bs - B), np.matmul(covariance_X, (Bs - B).transpose()))
    
    return np.trace(covariance_matrix) - 1 * np.linalg.norm(P, 'f')

print(expected_cost(As, Ps))

1.2679491924311228


In [523]:
p_coefs = np.ones(p_parms)
permutation_list = itertools.permutations(np.identity(n))
permutation_list = np.array(list(permutation_list))

total = sum(p * c for p, c in zip(p_coefs, permutation_list))
print(total)

[[2. 2. 2.]
 [2. 2. 2.]
 [2. 2. 2.]]


In [524]:
import itertools

permutation_list = itertools.permutations(np.identity(n))
permutation_list = np.array(list(permutation_list))

def expected_cost_opt(variables):
    # create A
    A = np.zeros((n, n))
    A[np.tril_indices(n)] = variables[:a_parms]
    
    # create P
    p_coefs = variables[a_parms:]
    P = sum(p * c for p, c in zip(p_coefs, permutation_list))
    
    ## compute expected cost
    # compute inverses
    P_inv = np.linalg.inv(P)
    Ps_inv = np.linalg.inv(Ps)
    
    # compute B
    B = np.matmul(P_inv, np.matmul(A, P))
    Bs = np.matmul(Ps_inv, np.matmul(As, Ps))
    
    # compute covariance of X
    covariance_X = np.matmul(np.linalg.inv(np.identity(n ** 2) - np.kron(Bs, Bs)), Sigma.reshape(n ** 2)).reshape((n, n))
    
    # compute covariance of X_{val} - X_{pred}
    covariance_matrix = Sigma + np.matmul((Bs - B), np.matmul(covariance_X, (Bs - B).transpose()))
    
    # return cost
    return np.trace(covariance_matrix) # - 0.1 * np.linalg.norm(P, 'f')

# # set A
A = np.tril((np.random.rand(n, n)))

# # set P
p_coefs = np.append(1.0, np.zeros(p_parms - 1))

# # define variables
variables = np.append(A[np.tril_indices(n)], p_coefs.flatten())

# # compute expected cost
expected_cost_opt(variables)

8.031458342920567

In [575]:
# coefficients must sum to one
def eq_cons(variables):
    p_coefs = variables[a_parms:]
    return sum(p_coefs) - 1

# coefficients must not be negative
def ineq_cons(variables):
    p_coefs = variables[a_parms:]
    return p_coefs - np.zeros(p_parms)

cons = [{'type': 'ineq', 'fun': ineq_cons}, {'type': 'eq', 'fun': eq_cons}]

In [552]:
scores = []

In [588]:
a = np.random.rand(5)
print(a / np.sum(a))

[0.15347904 0.12755623 0.33138101 0.05707333 0.3305104 ]


In [583]:
from scipy import optimize



for i in range(1):
    results = optimize.minimize(expected_cost_opt, np.random.rand(a_parms + p_parms), constraints = cons)
    p_coefs = results.x[a_parms:]
    p_perm = np.array(sum(p * c for p, c in zip(p_coefs, permutation_list)))
    print(np.round(p_perm, 2))
    print(closest_perm(p_perm))
    scores.append(np.sum(closest_perm(p_perm) == Ps) == n ** 2)
    print(sum(scores) / len(scores))
    print(results.fun)

print()
print(sum(scores) / len(scores))

[[ 0.01  0.99  0.01]
 [ 0.22  0.01  0.76]
 [ 0.77 -0.    0.23]]
[[0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]]
0.3684210526315789
3.01749985809515

0.3684210526315789


In [160]:
def closest_perm(P_DS):
    # convert to get the maximum linear assignment problem
    P_MOD = np.ones((n, n)) * np.max(P_DS) - P_DS
    
    # apply the maximum linear assignment algorithm
    row_ind, col_ind = optimize.linear_sum_assignment(P_MOD)

    # initialize Permutation matrix to return
    P_perm = np.zeros((n, n))

    # fill in permutation matrix
    for row, col in zip(row_ind, col_ind):
        P_perm[row][col] = 1 
    
    # re
    return P_perm

In [584]:
# get gradient of our outcome
def B_grad_a(A, P, i, j, As = As, Ps = Ps):
    P_inv = np.linalg.inv(P)
    Ps_inv = np.linalg.inv(Ps)
    
    B = np.matmul(P_inv, np.matmul(A, P))
    Bs = np.matmul(Ps_inv, np.matmul(As, Ps))
    
    J = np.zeros((n, n))
    J[i][j] = 1
    
    covariance_X = np.matmul(np.linalg.inv(np.identity(n ** 2) - np.kron(Bs, Bs)), Sigma.reshape(n ** 2)).reshape((n, n))

    return -2 * np.trace(np.matmul(covariance_X, np.matmul((Bs - B).transpose(), np.matmul(P_inv, np.matmul(J, P)))))

def B_grad_p(A, P, i, j, As = As, Ps = Ps):
    P_inv = np.linalg.inv(P)
    Ps_inv = np.linalg.inv(Ps)
    
    B = np.matmul(P_inv, np.matmul(A, P))
    Bs = np.matmul(Ps_inv, np.matmul(As, Ps))
    
    J = np.zeros((n, n))
    J[i][j] = 1
    
    covariance_X = np.matmul(np.linalg.inv(np.identity(n ** 2) - np.kron(Bs, Bs)), Sigma.reshape(n ** 2)).reshape((n, n))

    B_grad = np.matmul(P_inv, np.matmul(A, J))
    B_grad -= np.matmul(P_inv, np.matmul(J, np.matmul(P_inv, np.matmul(A, P))))
    
    return -2 * np.trace(np.matmul(covariance_X, np.matmul((Bs - B).transpose(), B_grad)))

A = np.zeros((n, n))
A[np.tril_indices(n)] = results.x[:6]
p_coefs = results.x[6:]
p_ds = np.array(sum(p * c for p, c in zip(p_coefs, permutation_list)))
print(f"A:\n{np.round(A, 2)}\n\nP:\n{np.round(p_ds, 2)}\n\n")

for i in range(n):
    for j in range(i + 1): 
        print(f"Grad(A[{i+1}][{j+1}]: {np.round(B_grad_a(A, p_ds, i, j), 2)}.")

print()

for i in range(n):
    for j in range(n):
        print(f"Grad(P[{i+1}][{j+1}]: {np.round(B_grad_p(A, p_ds, i, j), 2)}.")

A:
[[ 0.88  0.    0.  ]
 [ 0.7   0.22  0.  ]
 [ 0.18  0.99 -0.25]]

P:
[[ 0.01  0.99  0.01]
 [ 0.22  0.01  0.76]
 [ 0.77 -0.    0.23]]


Grad(A[1][1]: -0.22.
Grad(A[2][1]: -0.0.
Grad(A[2][2]: -0.0.
Grad(A[3][1]: 0.23.
Grad(A[3][2]: 0.3.
Grad(A[3][3]: 0.22.

Grad(P[1][1]: 0.37.
Grad(P[1][2]: 0.04.
Grad(P[1][3]: 0.03.
Grad(P[2][1]: 0.39.
Grad(P[2][2]: 0.22.
Grad(P[2][3]: 0.27.
Grad(P[3][1]: -0.36.
Grad(P[3][2]: -0.25.
Grad(P[3][3]: -0.28.


## Gradient for cost function
$$C(A, P) = \text{Tr}\left((B^* - B)\Sigma_X(B^* - B)\right)$$

$$P = \sum_{i=1}^{n!}\lambda_i P_i$$

$$\frac{\partial C(A, P)}{\partial lambda_{i}} = \frac{\partial C(A, P)}{\partial P